# Interactivity

This notebook walks through a simple, yet powerful example of how our abstract interactivity system can be used to link graphics and their features together.

In [1]:
import fastplotlib as fpl
from sidecar import Sidecar
import numpy as np
from ipywidgets import VBox

Load in some neural data.

In [2]:
# reconstructed movie of CaImAn Sue demo movie after running motion correction and CNMF algorithms
movie = np.load('/home/clewis7/Desktop/rcm.npy')

# temporal traces of identified neurons
temporal = np.load('/home/clewis7/Desktop/temporal.npy')

# contours of identified neurons
contours = np.load('/home/clewis7/Desktop/contours.npy', allow_pickle=True)

Will need a euclidean helper function to indentify which contours has been clicked on. We hope to soon include this, and other common callback functions, in our interactivity system :D

In [57]:
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

### Plot creation

Create an `ImageWidget`to display the reconstructed movie and identified neurons. Also create a `Plot` instance to display the associated temporal data for each identified nueron.

In [58]:
# for the image data and contours
cnmf_iw = fpl.ImageWidget(movie, vmin_vmax_sliders=True, cmap="gnuplot2")

# add contours to the plot within the widget
contours_graphic = cnmf_iw.gridplot[0,0].add_line_collection(contours, colors="cyan", name="contours")

# temporal plot
plot_temporal = fpl.Plot(size=(600,100))

temporal_graphic = plot_temporal.add_line_collection(temporal, colors="magenta", name="temporal")

# a vertical line that is syncronized to the image widget "t" (timepoint) slider
def update_timepoint(ev):
    ix = ev.pick_info["selected_index"][0]
    cnmf_iw.sliders["t"].value = ix

selector = temporal_graphic.add_linear_selector()
selector.selection.add_event_handler(update_timepoint)

# stack the plots and show them in sidecar
sc = Sidecar(title="interactivity")

with sc:
    display(VBox([cnmf_iw.show(), plot_temporal.show()]))

RFBOutputContext()

RFBOutputContext()

### Interacitvity of `Graphics` using `link()`

In [59]:
# so we can view them one by one, first hide all of them
temporal_graphic[:].present = False
    
# link image to contours
image_graphic = cnmf_iw.gridplot[0,0]["image_widget_managed"]

image_graphic.link(
    "click",
    target=contours_graphic,
    feature="colors", 
    new_data="w", 
    callback=euclidean
)

# link contour color changes (which are triggered by the click events as defined above) to everything else

# thickness of contour
contours_graphic.link("colors", target=contours_graphic, feature="thickness", new_data=3)

# toggle temporal component when contour changes color
contours_graphic.link("colors", target=temporal_graphic, feature="present", new_data=True)

# autoscale temporal plot to the current temporal component
temporal_graphic[:].present.add_event_handler(plot_temporal.auto_scale)

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:143: UserWarning: Event handler <bound method Interaction._event_handler of 'contours': LineCollection @ 0x7febe751ea10
Collection of <155> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")


In [60]:
sc.close()

## Heatmap

In [39]:
# create plot
plot = fpl.GridPlot(shape=(1,2), names=[["heatmap", "traces"]])

# add temporal traces as heatmap
heatmap = plot[0,0].add_heatmap(temporal)

# add temporal traces as line stack
temporal_stack = plot[0,1].add_line_stack(temporal, colors="magenta")

# add linear selector to heatmap
selector = heatmap.add_linear_region_selector()

sc = Sidecar(title="heatmap interactive")

with sc:
    display(plot.show())

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### interactivity using event handlers

In [40]:
# define event handler
def color_change(ev):
    selected_indices = ev.pick_info["selected_indices"]
    
    for i in range(len(temporal_stack.graphics)):
        temporal_stack.graphics[i].colors = "magenta"
        temporal_stack.graphics[i].colors[selected_indices] = "white"

# add event handler
selector.selection.add_event_handler(color_change)

In [41]:
sc.close()